In [18]:
# Cell 1: Install minimal deps (safe re-run)
import sys, subprocess

def pip_install(pkg):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])
    except Exception as e:
        print(f"Warning: couldn't install {pkg}: {e}")

# Colab thường đã có pandas & python-dateutil; ta chỉ đảm bảo pydantic v2+
pip_install("pydantic>=2.0.0")
# Nếu môi trường thiếu dateutil (hiếm), bỏ comment dòng dưới:
# pip_install("python-dateutil>=2.8.2")

print("✅ Cell 1 done: Dependencies checked/installed.")


✅ Cell 1 done: Dependencies checked/installed.


thiết lập múi giờ, import

In [19]:
# Cell 2: Env & imports
import os
os.environ["TZ"] = "Asia/Ho_Chi_Minh"

# Core imports
import json
from datetime import datetime, date, timedelta
import pandas as pd
from dateutil.relativedelta import relativedelta
from dateutil import tz
from pydantic import BaseModel, Field, ValidationError

# Confirm timezone
local_tz = tz.gettz("Asia/Ho_Chi_Minh")
now_local = datetime.now(local_tz)
print(f"✅ Cell 2 done: Timezone set to Asia/Ho_Chi_Minh. Now: {now_local:%Y-%m-%d %H:%M:%S %Z}")


✅ Cell 2 done: Timezone set to Asia/Ho_Chi_Minh. Now: 2025-09-11 02:35:23 +07


cấu hình kết nối Postgres, helper db_available().

In [20]:
# DB config (PostgreSQL) for persona notebook
import os
from sqlalchemy import create_engine, text

PG_HOST = os.getenv("PG_HOST", "localhost")
PG_PORT = int(os.getenv("PG_PORT", "5435"))
PG_DB   = os.getenv("PG_DB", "db_fin")
PG_USER = os.getenv("PG_USER", "HiepData")
PG_PASSWORD = os.getenv("PG_PASSWORD", "123456")

PG_URL = f"postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DB}"

engine = None
try:
    engine = create_engine(PG_URL, pool_pre_ping=True, future=True)
    with engine.connect() as conn:
        conn.execute(text("SELECT 1"))
    print("✅ PostgreSQL connected (persona):", PG_URL.rsplit("@",1)[-1])
except Exception as e:
    engine = None
    print("ℹ️ Persona notebook cannot connect PostgreSQL:", e)


def db_available() -> bool:
    return engine is not None



✅ PostgreSQL connected (persona): localhost:5435/db_fin


đặt DATA_DIR để đọc CSV fallback khi không có DB.

In [21]:
# Load demo feature_monthly ưu tiên từ PostgreSQL, fallback CSV/JSON
import os, json, pandas as pd
from sqlalchemy import text

# Đảm bảo DATA_DIR tồn tại trước khi dùng
if 'DATA_DIR' not in globals():
    DATA_DIR = os.getcwd()

csv_path = os.path.join(DATA_DIR, "features_monthly.csv")
json_path = os.path.join(DATA_DIR, "feature_monthly_demo_updated.json")

df = None
if db_available():
    try:
        with engine.connect() as conn:
            sql = text("""
                SELECT * FROM features_monthly
                ORDER BY customer_id ASC
                LIMIT 1
            """)
            df = pd.read_sql(sql, conn)
            print("✅ Loaded demo row from PostgreSQL")
    except Exception as e:
        print("ℹ️ DB read failed, fallback CSV/JSON:", e)

if df is None or df.empty:
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        print(f"✅ Loaded CSV: {csv_path}")
    elif os.path.exists(json_path):
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        df = pd.DataFrame(data)
        print(f"✅ Loaded JSON: {json_path}")
    else:
        raise FileNotFoundError("❌ Không tìm thấy dữ liệu demo (DB/CSV/JSON).")

print("---- DataFrame preview ----")
display(df)



✅ Loaded demo row from PostgreSQL
---- DataFrame preview ----


,customer_id,year_month,age,segment,income,spend,balance_avg,loan,digital_logins_30d,incoming_tx_cnt_30d,...,spend_ratio,dti,cashflow_volatility,liquidity_buffer,digital_index,inflow_baseline_90d,max_inflow_z_30d,max_inflow_pct_30d,large_inflow_flag_7d,days_since_large_inflow
0,1,2025-08,48,family,9475780.0,7623236.0,1877863.0,20676745.0,19,2,...,0.804,2.182,0.164,0.868,0.826,9818341.0,-0.016,0.496,0,119


In [22]:
# Cell 3: Local path config (run outside Colab)
import os

DATA_DIR = os.getcwd()
print("✅ Using local DATA_DIR:", DATA_DIR)


✅ Using local DATA_DIR: c:\Users\admin1\Downloads\hackathon


Install & init Gemini

In [23]:
# # Cell 4: Load demo feature_monthly từ file local trong thư mục dự án

# import os, json, pandas as pd

# csv_path = os.path.join(DATA_DIR, "features_monthly.csv")
# json_path = os.path.join(DATA_DIR, "feature_monthly_demo_updated.json")  # optional fallback nếu có

# if os.path.exists(csv_path):
#     df = pd.read_csv(csv_path)
#     print(f"✅ Loaded CSV: {csv_path}")
# elif os.path.exists(json_path):
#     with open(json_path, "r", encoding="utf-8") as f:
#         data = json.load(f)
#     df = pd.DataFrame(data)
#     print(f"✅ Loaded JSON: {json_path}")
# else:
#     raise FileNotFoundError(
#         "❌ Không tìm thấy 'features_monthly.csv' trong thư mục dự án."
#     )

# print("---- DataFrame preview ----")
# display(df)


In [24]:
# Cell 5: Build planning_context JSON từ 1 hàng feature_monthly

from datetime import datetime, timedelta
from dateutil import tz

def _to_int_safe(x, default=0):
    try:
        return int(x)
    except Exception:
        return default

def _parse_bills(s: str):
    """
    '10:rent=3500000;15:electric=500000;25:net=300000'
    -> [{'dom':10,'title':'rent','amount':3500000}, ...]
    """
    out = []
    if not isinstance(s, str) or not s.strip():
        return out
    parts = [p.strip() for p in s.split(";") if p.strip()]
    for p in parts:
        try:
            dom_str, rest = p.split(":", 1)
            title, amount_str = rest.split("=", 1)
            out.append({
                "dom": int(dom_str),
                "title": title.strip(),
                "amount": _to_int_safe(amount_str)
            })
        except:
            continue
    return out

def _parse_events(s: str):
    """
    '13:birthday=300000;20:trip=1000000'
    -> [{'dom':13,'title':'birthday','budget':300000}, ...]
    """
    out = []
    if not isinstance(s, str) or not s.strip():
        return out
    parts = [p.strip() for p in s.split(";") if p.strip()]
    for p in parts:
        try:
            dom_str, rest = p.split(":", 1)
            title, amount_str = rest.split("=", 1)
            out.append({
                "dom": int(dom_str),
                "title": title.strip(),
                "budget": _to_int_safe(amount_str)
            })
        except:
            continue
    return out

def _date_candidates_in_horizon(today_dt, dom_values, horizon_days):
    """
    Trả về danh sách datetime trong cửa sổ [today, today+horizon_days)
    khớp với day-of-month (DOM) cho tháng hiện tại.
    Nếu DOM đã qua trong tháng hiện tại, bỏ qua (không lùi về tháng trước).
    """
    last_dt = today_dt + timedelta(days=horizon_days)
    out = []
    cur = today_dt
    while cur < last_dt:
        if cur.day in dom_values and cur.month == today_dt.month:
            out.append(cur)
        cur += timedelta(days=1)
    return out

def build_planning_context_from_row(row, horizon_days=7, persona="mentor"):
    local_tz = tz.gettz("Asia/Ho_Chi_Minh")
    today_dt = datetime.now(local_tz).date()  # dùng giờ địa phương đã set

    # Đọc trường bắt buộc (ánh xạ từ schema thực tế)
    customer_id = int(row.get("customer_id", 0))

    # income/fixed/variable: ánh xạ mềm từ các cột có thật
    income_net = int(row.get("income_net_month", row.get("income", 0)))
    fixed_bills = int(row.get("fixed_bills_month", row.get("loan", 0)))
    variable_spend = int(row.get("variable_spend_month", row.get("spend", 0)))

    cash_on_hand = int(row.get("cash_on_hand", 0))

    # goal: nếu không có, đặt mục tiêu tối thiểu để không gãy luồng
    goal_amount = int(row.get("goal_amount", max(0, int(0.1 * income_net) if income_net else 1000000)))
    # deadline mặc định = cuối tháng hiện tại
    try:
        goal_deadline = str(row.get("goal_deadline"))
        if goal_deadline in [None, "None", "nan", "NaT"]:
            raise Exception()
    except Exception:
        from datetime import date
        d = date(today_dt.year, today_dt.month, 28)
        goal_deadline = d.isoformat()

    # Budgets baseline theo nhóm (monthly) với fallback về spend/4
    def _fallback_budget(key):
        return int(row.get(key, 0)) if row.get(key) is not None else 0

    base_guess = int(variable_spend/4) if variable_spend else 0
    budgets_baseline = {
        "food_out": _fallback_budget("food_out_month") or base_guess,
        "snacks": _fallback_budget("snacks_month") or base_guess,
        "transport": _fallback_budget("transport_month") or base_guess,
        "entertainment": _fallback_budget("entertainment_month") or base_guess,
    }

    # Payday DOM (cho phép nhiều giá trị phân tách ; hoặc ,)
    payday_dom_str = str(row.get("payday_dom", "")).strip()
    payday_dom_list = []
    if payday_dom_str:
        for token in payday_dom_str.replace(",", ";").split(";"):
            token = token.strip()
            if token:
                try:
                    payday_dom_list.append(int(token))
                except:
                    pass

    # Bills & Events
    bills = _parse_bills(str(row.get("bill_dom_list", "")))
    events = _parse_events(str(row.get("events_dom_list", "")))

    # Tìm các ngày đặc biệt trong horizon
    # Payday
    special_days = []
    if payday_dom_list:
        paydays = _date_candidates_in_horizon(today_dt, payday_dom_list, horizon_days)
        for d in paydays:
            special_days.append({"date": d.isoformat(), "kind": "payday"})

    # Bills
    bill_doms = [b["dom"] for b in bills]
    bill_dates = _date_candidates_in_horizon(today_dt, bill_doms, horizon_days)
    for d in bill_dates:
        # khớp bản ghi
        dom = d.day
        for b in bills:
            if b["dom"] == dom:
                special_days.append({
                    "date": d.isoformat(),
                    "kind": "bill",
                    "title": b["title"],
                    "amount_lock": b["amount"]
                })

    # Events
    event_doms = [e["dom"] for e in events]
    event_dates = _date_candidates_in_horizon(today_dt, event_doms, horizon_days)
    for d in event_dates:
        dom = d.day
        for e in events:
            if e["dom"] == dom:
                special_days.append({
                    "date": d.isoformat(),
                    "kind": "event",
                    "title": e["title"],
                    "budget_est": e["budget"]
                })

    planning_context = {
        "customer_id": customer_id,
        "today": today_dt.isoformat(),
        "horizon_days": int(horizon_days),
        "income_net": income_net,
        "fixed_bills": fixed_bills,
        "variable_spend": variable_spend,
        "cash_on_hand": cash_on_hand,
        "goal_amount": goal_amount,
        "goal_deadline": goal_deadline,
        "budgets_baseline": budgets_baseline,
        "special_days": sorted(special_days, key=lambda x: x["date"]),
        "persona": persona  # "mentor" | "angry_mom" | "banter"
    }
    print("✅ planning_context generated.")
    return planning_context

# Lấy hàng đầu tiên để demo
row0 = df.iloc[0].to_dict()
planning_context_7d = build_planning_context_from_row(row0, horizon_days=14, persona="mentor")

import json
print(json.dumps(planning_context_7d, indent=2, ensure_ascii=False))


✅ planning_context generated.
{
  "customer_id": 1,
  "today": "2025-09-11",
  "horizon_days": 14,
  "income_net": 9475780,
  "fixed_bills": 20676745,
  "variable_spend": 7623236,
  "cash_on_hand": 0,
  "goal_amount": 947578,
  "goal_deadline": "2025-09-28",
  "budgets_baseline": {
    "food_out": 1905809,
    "snacks": 1905809,
    "transport": 1905809,
    "entertainment": 1905809
  },
  "special_days": [],
  "persona": "mentor"
}


In [25]:
# Cell 6: Install & init Gemini (env-based, safe when missing)
import os, sys, subprocess

def pip_install(pkg):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])
    except Exception as e:
        print(f"Warning: couldn't install {pkg}: {e}")

# Đảm bảo có SDK chính thức của Gemini
pip_install("google-generativeai>=0.7.2")

# Đọc API key từ biến môi trường (đã thiết lập ngoài notebook)
# Nếu kernel không thấy biến môi trường do khởi động trước khi `setx`, thử lấy từ Windows User env (Registry)
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY", "")
if not GEMINI_API_KEY:
    try:
        import sys
        if sys.platform.startswith("win"):
            import winreg
            with winreg.OpenKey(winreg.HKEY_CURRENT_USER, "Environment") as k:
                v, _ = winreg.QueryValueEx(k, "GEMINI_API_KEY")
                if v:
                    os.environ["GEMINI_API_KEY"] = v
                    GEMINI_API_KEY = v
                    print("ℹ️ Loaded GEMINI_API_KEY from Windows user env.")
    except Exception:
        pass

GEMINI_MODEL = os.environ.get("GEMINI_MODEL", "gemini-2.0-flash")

try:
    import google.generativeai as genai
except Exception:
    genai = None

model = None
if genai is not None and GEMINI_API_KEY:
    try:
        genai.configure(api_key=GEMINI_API_KEY)
        model = genai.GenerativeModel(GEMINI_MODEL)
        print(f"✅ Cell 6 done: Gemini model '{GEMINI_MODEL}' is ready.")
    except Exception as e:
        print("⚠️ Gemini init failed:", e)
else:
    print("ℹ️ Gemini not configured (missing package or GEMINI_API_KEY). LLM calls will be skipped.")


✅ Cell 6 done: Gemini model 'gemini-2.0-flash' is ready.


In [26]:
# # Cell 7 (REPLACE): Generate strict-JSON 7-day plan from planning_context

# import json, re

# # Re-init model with strict JSON response (only if model is available)
# PERSONA_MAP = {
#     "mentor": "Người cố vấn thông thái",
#     "angry_mom": "Mẹ giận dữ",
#     "banter": "Thích cà khịa"
# }

# def build_user_prompt(pc: dict) -> str:
#     persona_label = PERSONA_MAP.get(pc.get("persona", "mentor"), "Người cố vấn thông thái")
#     return json.dumps({
#         "persona": persona_label,
#         "planning_context": pc,
#         "output_contract": {
#             "week_plan_item": {"date": "YYYY-MM-DD", "tasks": ["task1", "task2"]},
#             "week_plan_length": pc.get("horizon_days", 7),
#             "supervision_note_example": "Tôi sẽ giám sát tuần này. Đạt → lặp lại; Không đạt → điều chỉnh."
#         }
#     }, ensure_ascii=False)

# def generate_week_plan_json(pc: dict):
#     if model is None:
#         raise RuntimeError("GEMINI_API_KEY chưa được thiết lập hoặc SDK chưa sẵn sàng. Không thể gọi LLM.")

#     # override system instruction via new GenerativeModel with JSON output
#     mdl = None
#     try:
#         import google.generativeai as genai
#         mdl = genai.GenerativeModel(
#             model_name=GEMINI_MODEL,
#             system_instruction=(
#                 "Bạn là LLM tư vấn tiết kiệm cá nhân.\n"
#                 "Nhiệm vụ:\n"
#                 "1. KHÔNG giải thích, KHÔNG viết thêm lời ngoài JSON.\n"
#                 '2. Chỉ xuất JSON hợp lệ có 2 key: "week_plan" (list) và "supervision_note".\n'
#                 '3. "week_plan" có đúng N ngày (N=horizon_days).\n'
#                 '   - Mỗi ngày: {"date":"YYYY-MM-DD","tasks":["task1","task2",...]}.\n'
#                 "   - Mỗi task phải là mệnh lệnh rõ ràng, có số cụ thể hoặc hành động đo lường được.\n"
#                 '   - Cấm dùng từ mơ hồ kiểu "xem xét", "theo dõi", "lên kế hoạch".\n'
#                 "4. Áp dụng special_days:\n"
#                 '   - payday → task "chuyển ngay 20–30% lương (hoặc số đủ để đạt weekly_save) vào quỹ", KHÔNG vượt 30%.\n'
#                 '   - bill → task "khóa số tiền bill X" + "cấm chi tiêu biến đổi trong ngày đó".\n'
#                 '   - event → task "phong bì X cho sự kiện" + "cắt giải trí về 0 trong tuần này".\n'
#                 "5. Persona ảnh hưởng đến CÁCH VIẾT task:\n"
#                 "   - Mentor: nhẹ nhàng nhưng rõ con số.\n"
#                 "   - Angry_mom: mệnh lệnh cứng rắn, nghiêm khắc.\n"
#                 "   - Banter: châm chọc, hóm hỉnh, nhưng vẫn nêu con số cụ thể.\n"
#                 '6. supervision_note luôn = "Tôi sẽ giám sát tuần này. Đạt → lặp lại; Không đạt → điều chỉnh."'
#             ),
#             generation_config={
#                 "temperature": 0.4,
#                 "response_mime_type": "application/json"
#             }
#         )
#     except Exception as e:
#         raise RuntimeError(f"Không khởi tạo được Gemini model: {e}")

#     prompt_payload = build_user_prompt(pc)
#     resp = mdl.generate_content(prompt_payload)
#     text = ""
#     if resp and getattr(resp, "candidates", None):
#         part = resp.candidates[0].content.parts[0]
#         text = getattr(part, "text", "") or getattr(part, "inline_data", {}).get("data", "")
#     if not text:
#         raise ValueError("Model returned empty response.")

#     # Parse JSON
#     try:
#         data = json.loads(text)
#     except json.JSONDecodeError:
#         m = re.search(r"\{[\s\S]*\}\s*$", text.strip())
#         if not m:
#             raise
#         data = json.loads(m.group(0))

#     if "week_plan" not in data or not isinstance(data["week_plan"], list):
#         raise AssertionError("week_plan missing or not a list")
#     if "supervision_note" not in data:
#         raise AssertionError("supervision_note missing")

#     expected_len = int(pc.get("horizon_days", 7))
#     if len(data["week_plan"]) != expected_len:
#         data["week_plan"] = data["week_plan"][:expected_len]

#     print("✅ Generated 7-day plan JSON:")
#     print(json.dumps(data, indent=2, ensure_ascii=False))
#     return data

# week_plan_json = generate_week_plan_json(planning_context_7d)


In [27]:
# # Cell 8: Generate 14-day plan JSON (có ngày event 13/09)

# # Tạo planning_context 14 ngày từ row đã load
# planning_context_14d = build_planning_context_from_row(
#     df.iloc[0].to_dict(),
#     horizon_days=14,
#     persona="mentor"   # bạn có thể đổi: "angry_mom", "banter"
# )

# week_plan_json_14d = generate_week_plan_json(planning_context_14d)


In [28]:
# # Cell 9 (updated): Persona-based reply với số tiền cụ thể

# import re

# def _extract_amount(text: str) -> int:
#     """
#     Parse số tiền từ event_text (dạng '100k', '200000', '1.2m', '1,000,000').
#     Trả về số int (VND) hoặc None nếu không parse được.
#     """
#     text = text.lower().replace(".", "").replace(",", "")
#     m = re.search(r"(\d+)(k|m|vnđ|vnd)?", text)
#     if not m:
#         return None
#     num, unit = m.group(1), m.group(2)
#     amount = int(num)
#     if unit == "k":
#         amount *= 1000
#     elif unit == "m":
#         amount *= 1000000
#     return amount

# def persona_reply(event_text: str, persona: str = "mentor") -> str:
#     """
#     Trả về phản hồi theo persona, luôn kèm số tiền điều chỉnh cụ thể nếu parse được.
#     """
#     amt = _extract_amount(event_text)
#     amt_str = f"{amt:,} VNĐ".replace(",", ".") if amt else None

#     if persona == "mentor":
#         if amt_str:
#             return f"Bạn vừa chi {amt_str} cho {event_text}. Hãy cắt đúng {amt_str} khỏi chi tiêu ngày mai để giữ mục tiêu."
#         else:
#             return f"Bạn vừa phát sinh: {event_text}. Hãy giảm chi tiêu ngày mai tương ứng để giữ mục tiêu."
#     elif persona == "angry_mom":
#         if amt_str:
#             return f"Lại tiêu {amt_str} hả? Ngay lập tức dừng lại! Ngày mai trừ thẳng {amt_str} khỏi ngân sách."
#         else:
#             return f"Chi tiêu vô tội vạ nữa hả? {event_text} là quá mức rồi! Ngay lập tức dừng lại."
#     elif persona == "banter":
#         if amt_str:
#             return f"Ối dào, {event_text} làm ví teo {amt_str} rồi! Thôi, hôm nay stop, mai bù lại {amt_str} nhé."
#         else:
#             return f"Ôi trời, {event_text} làm ví bạn méo rồi! Hôm nay stop, mai bù lại nha."
#     else:
#         return f"[Unknown persona] {event_text}"

# # Demo
# print("Mentor:", persona_reply("Uống trà sữa 100k", "mentor"))
# print("Angry mom:", persona_reply("Ăn buffet 250000 VNĐ", "angry_mom"))
# print("Banter:", persona_reply("Mua áo mới 1m2", "banter"))


In [29]:
# # Cell 10: Demo hội thoại end-to-end (mock user)

# # 1. Người dùng chọn persona
# chosen_persona = "banter"  # "mentor", "angry_mom", hoặc "banter"
# print(f"👤 Người dùng chọn persona: {chosen_persona}")

# # 2. Sinh kế hoạch (7 hoặc 14 ngày)
# horizon = 14  # đổi thành 7 nếu chỉ muốn 1 tuần
# planning_context_demo = build_planning_context_from_row(
#     df.iloc[0].to_dict(),
#     horizon_days=horizon,
#     persona=chosen_persona
# )
# plan_json = generate_week_plan_json(planning_context_demo)

# # In kế hoạch gọn gàng
# print("\n📅 Kế hoạch sinh ra:")
# for day in plan_json["week_plan"]:
#     date = day["date"]
#     tasks = " | ".join(day["tasks"])
#     print(f"{date}: {tasks}")

# print("\n🔒 supervision_note:", plan_json["supervision_note"])

# # 3. Người dùng phát sinh chi tiêu
# event_text = "Uống trà sữa 100k"
# print(f"\n👤 Người dùng: {event_text}")

# # 4. Persona phản hồi
# reply = persona_reply(event_text, chosen_persona)
# print(f"🤖 {chosen_persona} reply:", reply)


In [30]:
# # Cell 11: Chat loop demo (tương tác như chat, thoát khi gõ "exit")

# import os

# def regenerate_plan(planning_context, feedback: str):
#     """Gọi lại model với feedback bổ sung để sinh kế hoạch mới"""
#     pc = planning_context.copy()
#     pc["user_feedback"] = feedback
#     return generate_week_plan_json(pc)

# def chat_loop():
#     # B1: chọn persona
#     persona = input("Chọn persona (mentor / angry_mom / banter): ").strip().lower()
#     if persona not in ["mentor", "angry_mom", "banter"]:
#         persona = "mentor"
#     print(f"✅ Persona đã chọn: {persona}")

#     # B2: chọn horizon
#     horizon = input("Bạn muốn kế hoạch mấy ngày? (7/14): ").strip()
#     horizon = 14 if horizon == "14" else 7

#     planning_context = build_planning_context_from_row(
#         df.iloc[0].to_dict(),
#         horizon_days=horizon,
#         persona=persona
#     )
#     plan_json = generate_week_plan_json(planning_context)

#     print("\n📅 Kế hoạch ban đầu:")
#     for day in plan_json["week_plan"]:
#         print(f"{day['date']}: {' | '.join(day['tasks'])}")
#     print("supervision_note:", plan_json["supervision_note"])
#     print("\n--- Bắt đầu chat (gõ 'exit' để thoát) ---")

#     # Loop chat
#     while True:
#         user_input = input("\n👤 Bạn: ").strip()
#         if user_input.lower() in ["exit", "quit"]:
#             print("🤖 LLM: Kết thúc chat. Hẹn gặp lại!")
#             break
#         elif user_input.lower() == "plan":
#             print("\n📅 Kế hoạch hiện tại:")
#             for day in plan_json["week_plan"]:
#                 print(f"{day['date']}: {' | '.join(day['tasks'])}")
#         elif user_input.lower().startswith("regen:"):
#             feedback = user_input[len("regen:"):].strip()
#             print(f"🤖 LLM: Đang sinh lại kế hoạch theo góp ý: '{feedback}'...")
#             plan_json = regenerate_plan(planning_context, feedback)
#             for day in plan_json["week_plan"]:
#                 print(f"{day['date']}: {' | '.join(day['tasks'])}")
#         else:
#             reply = persona_reply(user_input, persona)
#             print(f"🤖 {persona}: {reply}")

# # Chạy chat loop (bỏ qua khi NON_INTERACTIVE=1)
# if os.environ.get("NON_INTERACTIVE", "0") != "1":
#     chat_loop()
# else:
#     print("ℹ️ NON_INTERACTIVE=1: bỏ qua chat loop.")


In [31]:
# Cell A: Session state & helpers

import re
from datetime import datetime
from dateutil import tz

# Khởi tạo "bộ nhớ hội thoại" cho phiên
session_state = {
    "persona": "mentor",              # "mentor" | "angry_mom" | "banter"
    "goal_reason": None,              # ví dụ: "đi du lịch Nha Trang"
    "spend_events_log": [],           # list[{"ts","text","amount","category"}]
    "running_overage_today": 0        # tổng vượt ngân sách (demo, có thể mở rộng)
}

def set_persona(persona: str):
    persona = (persona or "").lower().strip()
    if persona not in ["mentor", "angry_mom", "banter"]:
        persona = "mentor"
    session_state["persona"] = persona
    return persona

def set_goal_reason(text: str | None):
    session_state["goal_reason"] = text.strip() if text else None

def _extract_amount(text: str) -> int | None:
    """
    Nhận các dạng phổ biến: 100k, 200000, 1.2m, 1m2, 150.000, 150,000, 150k…
    Trả về số VND (int) hoặc None nếu không parse được.
    """
    t = text.lower().replace("vnđ", "").replace("vnd", "")
    # chuẩn hóa: bỏ dấu chấm/ngăn cách
    t_norm = t.replace(".", "").replace(",", "")
    # bắt '1m2' -> 1.2m
    t_norm = re.sub(r"(\d)tr(\d)", r"\1.\2m", t_norm)  # 1tr2 -> 1.2m
    # tìm số + đơn vị
    m = re.search(r"(\d+(?:\.\d+)?)(m|k)?", t_norm)
    if not m:
        return None
    num, unit = m.group(1), m.group(2)
    try:
        if unit == "m":
            return int(float(num) * 1_000_000)
        if unit == "k":
            return int(float(num) * 1_000)
        return int(float(num))
    except:
        return None

def _guess_category(text: str) -> str:
    text = text.lower()
    if any(k in text for k in ["trà sữa", "snack", "ăn vặt", "bánh", "kẹo"]):
        return "snacks"
    if any(k in text for k in ["ăn ngoài", "cafe", "cà phê", "nhà hàng", "mì cay", "bún", "phở"]):
        return "food_out"
    if any(k in text for k in ["taxi", "grab", "bus", "xe buýt", "xăng", "trạm thu phí"]):
        return "transport"
    if any(k in text for k in ["phim", "game", "nhạc", "karaoke", "giải trí"]):
        return "entertainment"
    if any(k in text for k in ["áo", "quần", "giày", "mua sắm", "phụ kiện"]):
        return "shopping"
    return "other"

def add_spend_event(user_text: str) -> dict:
    """
    Ghi nhận một lần chi tiêu phát sinh vào session_state['spend_events_log'].
    Trả về bản ghi đã thêm.
    """
    amt = _extract_amount(user_text)
    cat = _guess_category(user_text)
    local_tz = tz.gettz("Asia/Ho_Chi_Minh")
    rec = {
        "ts": datetime.now(local_tz).isoformat(timespec="seconds"),
        "text": user_text,
        "amount": amt,
        "category": cat
    }
    session_state["spend_events_log"].append(rec)
    # (demo) cộng dồn overage nếu có số
    if amt:
        session_state["running_overage_today"] += amt
    return rec

print("✅ Cell A ready: session_state & helpers initialized.")


✅ Cell A ready: session_state & helpers initialized.


In [32]:
# Cell B: LLM-driven persona reply (natural chat + numeric adjustment)

import json

# Map nhãn persona để đưa vào system prompt
PERSONA_MAP = {
    "mentor": "Người cố vấn thông thái — điềm tĩnh, rõ ràng, đưa số cụ thể.",
    "angry_mom": "Mẹ giận dữ — nghiêm khắc, thẳng thừng, KHÔNG xúc phạm cá nhân.",
    "banter": "Thích cà khịa — hóm hỉnh, châm chọc nhẹ, nhưng vẫn đưa số cụ thể."
}

def _short_special_days(pc: dict, max_days: int = 4) -> str:
    sdays = pc.get("special_days", [])[:max_days]
    return ", ".join([f"{d['date']}:{d['kind']}" for d in sdays]) or "không"

def _recent_spends(state: dict, k: int = 3) -> list:
    return state.get("spend_events_log", [])[-k:]

def llm_persona_reply(user_text: str, state: dict, planning_context: dict, plan_json: dict):
    """
    Gọi Gemini để sinh lời đáp hội thoại tự nhiên theo persona + điều chỉnh số cụ thể.
    Output JSON:
    {
      "message": "<1–2 câu>",
      "adjustment": {"amount": <int or null>, "category": "<cat or null>", "when": "today|tomorrow"}
    }
    """
    # Nếu thiếu model (chưa cấu hình GEMINI_API_KEY), trả fallback an toàn
    try:
        import google.generativeai as genai
    except Exception:
        genai = None
    if genai is None or os.environ.get("GEMINI_API_KEY", "") == "":
        # Fallback: đưa ra khuyến nghị deterministic dựa trên parsing số tiền
        new_rec = add_spend_event(user_text)
        amt = new_rec.get("amount")
        cat = new_rec.get("category") or "other"
        msg = (
            f"Ghi nhận chi tiêu '{user_text}'. "
            + (f"Ngày mai trừ {amt:,} VNĐ khỏi {cat}.".replace(",", ".") if amt else "Hãy nêu số tiền để tôi trừ chính xác ngày mai.")
        )
        return {"message": msg, "adjustment": {"amount": amt, "category": cat, "when": "tomorrow"}}

    persona = state.get("persona", "mentor")
    persona_desc = PERSONA_MAP.get(persona, PERSONA_MAP["mentor"])
    goal_reason = state.get("goal_reason") or "mục tiêu tiết kiệm đã thiết lập"
    goal_deadline = planning_context.get("goal_deadline")
    horizon = planning_context.get("horizon_days", 7)
    special_days_short = _short_special_days(planning_context)
    recent = _recent_spends(state, 3)

    # Đưa spend mới vào log (nếu chưa)
    new_rec = add_spend_event(user_text)

    system_instruction = (
        "Bạn là trợ lý tài chính theo persona. Hãy trả lời HỘI THOẠI tự nhiên (1–2 câu) theo phong cách sau:\n"
        f"- {persona_desc}\n\n"
        "Ràng buộc:\n"
        "1) Luôn chốt 1 mệnh lệnh hành động với CON SỐ cụ thể (ví dụ: 'mai trừ 200.000 khỏi snacks; hôm nay dừng ăn vặt').\n"
        "2) Cấm xúc phạm cá nhân, tránh công kích thân thể. Có thể nghiêm khắc/hóm hỉnh nhưng lịch sự.\n"
        "3) Nếu không trích được số tiền từ lời người dùng, đề nghị họ nêu số hoặc đưa mức trần an toàn (≤100.000 VNĐ).\n"
        "4) Trả về JSON duy nhất theo schema sau (không thêm chữ nào ngoài JSON):\n"
        '{ "message":"<1-2 câu>", "adjustment":{"amount": <số hoặc null>, "category":"<chuỗi hoặc null>", "when":"today|tomorrow"} }'
    )

    user_payload = {
        "user_text": user_text,
        "goal_reason": goal_reason,
        "goal_deadline": goal_deadline,
        "horizon_days": horizon,
        "special_days": special_days_short,
        "recent_spends": recent,
        "new_spend": new_rec,
        "plan_hint": plan_json.get("week_plan", [])[:2]
    }

    model_json = genai.GenerativeModel(
        model_name=os.environ.get("GEMINI_MODEL", "gemini-2.0-flash"),
        system_instruction=system_instruction,
        generation_config={
            "temperature": 0.6,
            "response_mime_type": "application/json"
        }
    )

    resp = model_json.generate_content(json.dumps(user_payload, ensure_ascii=False))
    text = resp.candidates[0].content.parts[0].text if resp and resp.candidates else ""
    try:
        out = json.loads(text)
    except json.JSONDecodeError as e:
        import re
        m = re.search(r"\{[\s\S]*\}\s*$", text.strip())
        if not m:
            raise e
        out = json.loads(m.group(0))

    if "adjustment" not in out:
        out["adjustment"] = {"amount": None, "category": None, "when": "today"}
    if out["adjustment"].get("category") in [None, "", "unknown"]:
        out["adjustment"]["category"] = new_rec.get("category") or "other"

    return out

print("✅ Cell B ready: llm_persona_reply() is available.")


✅ Cell B ready: llm_persona_reply() is available.


In [33]:
# # Ví dụ thử nhanh (ngoài chat loop)
# set_persona("angry_mom")
# set_goal_reason("đi du lịch Nha Trang")
# reply = llm_persona_reply("nay thèm đồ ngọt quá, lỡ mua 200k bánh rồi", session_state, planning_context_14d, week_plan_json_14d)
# reply


In [ ]:
# Cell UI: Chat demo bằng ipywidgets (3 nút persona + chatbox)
import os, json
import ipywidgets as w
from IPython.display import display, clear_output

# Trạng thái UI
ui_state = {
    "persona": session_state.get("persona", "mentor"),
    "horizon": 7,
    "pc": None,
    "plan": None,
}

# Nút chọn persona
btn_mentor = w.Button(description="Mentor", button_style="info")
btn_angry  = w.Button(description="Angry mom", button_style="warning")
btn_banter = w.Button(description="Banter", button_style="success")

# Chọn horizon
horizon_dd = w.Dropdown(options=[7,14], value=7, description="Horizon")

# Nút khởi tạo kế hoạch
btn_init = w.Button(description="Khởi tạo kế hoạch", button_style="primary")

# Khu vực log và chat
out_plan = w.Output(layout={"border":"1px solid #ccc"})
chat_input = w.Text(placeholder="Nhập tin nhắn (vd: Uống trà sữa 100k)")
btn_send = w.Button(description="Gửi", button_style="primary")
chat_log = w.Output(layout={"border":"1px solid #ccc", "height":"250px", "overflow":"auto"})

# Handlers

def _set_persona(p):
    set_persona(p)
    ui_state["persona"] = p
    with chat_log:
        print(f"[system] persona = {p}")

def _on_click_persona(b):
    if b is btn_mentor:
        _set_persona("mentor")
    elif b is btn_angry:
        _set_persona("angry_mom")
    else:
        _set_persona("banter")

btn_mentor.on_click(_on_click_persona)
btn_angry.on_click(_on_click_persona)
btn_banter.on_click(_on_click_persona)


def _init_plan(_):
    ui_state["horizon"] = int(horizon_dd.value)
    # Tạo planning context từ hàng đầu tiên df
    pc = build_planning_context_from_row(df.iloc[0].to_dict(), horizon_days=ui_state["horizon"], persona=ui_state["persona"])
    ui_state["pc"] = pc
    try:
        plan = generate_week_plan_json(pc)
    except Exception as e:
        plan = {"week_plan": [], "supervision_note": f"LLM unavailable: {e}"}
    ui_state["plan"] = plan
    with out_plan:
        clear_output()
        print("📅 Kế hoạch:")
        for day in plan.get("week_plan", [])[:ui_state["horizon"]]:
            print(f"{day['date']}: {' | '.join(day['tasks'])}")
        print("\n🔒 supervision_note:", plan.get("supervision_note"))

btn_init.on_click(_init_plan)


def _send(_):
    text = chat_input.value.strip()
    if not text:
        return
    chat_input.value = ""
    with chat_log:
        print(f"👤 Bạn: {text}")
    pc = ui_state.get("pc")
    plan = ui_state.get("plan") or {"week_plan": []}
    if not pc:
        with chat_log:
            print("🤖 LLM: Hãy khởi tạo kế hoạch trước (nhấn 'Khởi tạo kế hoạch').")
        return
    try:
        resp = llm_persona_reply(text, session_state, pc, plan)
        msg = resp.get("message") or "(no message)"
        with chat_log:
            print(f"🤖 {ui_state['persona']}: {msg}")
    except Exception as e:
        with chat_log:
            print(f"🤖 LLM error: {e}")

btn_send.on_click(_send)

# Layout
persona_box = w.HBox([w.HTML("<b>Persona:</b>&nbsp;"), btn_mentor, btn_angry, btn_banter])
init_box = w.HBox([horizon_dd, btn_init])
input_box = w.HBox([chat_input, btn_send])

ui = w.VBox([
    persona_box,
    init_box,
    w.HTML("<hr>"),
    w.HTML("<b>Kế hoạch</b>"),
    out_plan,
    w.HTML("<b>Chat</b>"),
    chat_log,
    input_box,
])

display(ui)
print("✅ UI chat sẵn sàng. Chọn persona, khởi tạo kế hoạch, rồi nhập tin nhắn.")



✅ UI chat sẵn sàng. Chọn persona, khởi tạo kế hoạch, rồi nhập tin nhắn.


In [35]:
# DDL: tạo bảng persona_plans, persona_plan_days, persona_chat_logs, persona_spend_events
from sqlalchemy import text

DDL_PERSONA = [
    """
    CREATE TABLE IF NOT EXISTS persona_plans (
      plan_id UUID PRIMARY KEY,
      customer_id INT NOT NULL,
      year_month VARCHAR(7),
      persona TEXT,
      horizon_days INT,
      goal_text TEXT,
      goal_amount NUMERIC(18,2),
      target_date DATE,
      weekly_save NUMERIC(18,2),
      feasibility TEXT,
      reasons JSONB,
      supervision_note TEXT,
      version INT DEFAULT 1,
      committed_by TEXT,
      created_at TIMESTAMPTZ DEFAULT now()
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS persona_plan_days (
      plan_id UUID NOT NULL REFERENCES persona_plans(plan_id) ON DELETE CASCADE,
      date DATE NOT NULL,
      tasks JSONB,
      PRIMARY KEY (plan_id, date)
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS persona_chat_logs (
      chat_id UUID PRIMARY KEY,
      customer_id INT,
      plan_id UUID NULL REFERENCES persona_plans(plan_id) ON DELETE SET NULL,
      ts TIMESTAMPTZ DEFAULT now(),
      role TEXT,
      text TEXT,
      payload JSONB
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS persona_spend_events (
      event_id UUID PRIMARY KEY,
      customer_id INT,
      ts TIMESTAMPTZ DEFAULT now(),
      text TEXT,
      amount NUMERIC(18,2),
      category TEXT,
      source TEXT,
      linked_plan_id UUID NULL REFERENCES persona_plans(plan_id) ON DELETE SET NULL
    )
    """,
    # Indexes
    "CREATE INDEX IF NOT EXISTS idx_persona_plans_cid_created ON persona_plans(customer_id, created_at DESC)",
    "CREATE INDEX IF NOT EXISTS idx_persona_plans_cid_ym ON persona_plans(customer_id, year_month)",
    "CREATE INDEX IF NOT EXISTS idx_persona_plan_days_plan ON persona_plan_days(plan_id)",
    "CREATE INDEX IF NOT EXISTS idx_persona_chat_logs_cid_ts ON persona_chat_logs(customer_id, ts DESC)",
    "CREATE INDEX IF NOT EXISTS idx_persona_chat_logs_plan_ts ON persona_chat_logs(plan_id, ts DESC)",
    "CREATE INDEX IF NOT EXISTS idx_persona_spends_cid_ts ON persona_spend_events(customer_id, ts DESC)",
    "CREATE INDEX IF NOT EXISTS idx_persona_spends_plan ON persona_spend_events(linked_plan_id)"
]

if db_available():
    with engine.begin() as conn:
        for stmt in DDL_PERSONA:
            conn.execute(text(stmt))
    print("✅ Persona tables ensured in PostgreSQL.")
else:
    print("ℹ️ Skip DDL: DB not available.")



✅ Persona tables ensured in PostgreSQL.


In [36]:
# Helpers: CRUD cho persona_plans, persona_plan_days, persona_chat_logs, persona_spend_events
import uuid
from typing import List, Dict, Any, Optional
from sqlalchemy import text

def _u():
    return str(uuid.uuid4())

def db_insert_persona_plan(conn, plan: Dict[str, Any], days: List[Dict[str, Any]]):
    """plan: dict các cột của persona_plans (không gồm created_at)
       days: list {date: YYYY-MM-DD, tasks: list[str]}
    """
    if "plan_id" not in plan:
        plan["plan_id"] = _u()
    plan_id = plan["plan_id"]
    # Insert plan
    cols = [k for k in plan.keys()]
    placeholders = ",".join([f":{k}" for k in cols])
    sql_plan = text(f"""
        INSERT INTO persona_plans({','.join(cols)})
        VALUES ({placeholders})
    """)
    conn.execute(sql_plan, plan)
    # Insert days
    if days:
        recs = [{"plan_id": plan_id, "date": d["date"], "tasks": json.dumps(d.get("tasks", []), ensure_ascii=False)} for d in days]
        sql_day = text("""
            INSERT INTO persona_plan_days(plan_id, date, tasks)
            VALUES (:plan_id, :date, CAST(:tasks AS JSONB))
            ON CONFLICT (plan_id, date) DO UPDATE SET tasks = EXCLUDED.tasks
        """)
        conn.execute(sql_day, recs)
    return plan_id

def db_upsert_chat(conn, customer_id: int, role: str, text_msg: str, payload: Optional[Dict[str, Any]] = None, plan_id: Optional[str] = None):
    rec = {
        "chat_id": _u(),
        "customer_id": customer_id,
        "plan_id": plan_id,
        "role": role,
        "text": text_msg,
        "payload": json.dumps(payload or {}, ensure_ascii=False)
    }
    sql = text("""
        INSERT INTO persona_chat_logs(chat_id, customer_id, plan_id, role, text, payload)
        VALUES (:chat_id, :customer_id, :plan_id, :role, :text, CAST(:payload AS JSONB))
    """)
    conn.execute(sql, rec)

def db_insert_spend_event(conn, customer_id: int, text_msg: str, amount: Optional[float], category: Optional[str], source: str = "user", plan_id: Optional[str] = None):
    rec = {
        "event_id": _u(),
        "customer_id": customer_id,
        "text": text_msg,
        "amount": amount,
        "category": category,
        "source": source,
        "linked_plan_id": plan_id
    }
    sql = text("""
        INSERT INTO persona_spend_events(event_id, customer_id, text, amount, category, source, linked_plan_id)
        VALUES (:event_id, :customer_id, :text, :amount, :category, :source, :linked_plan_id)
    """)
    conn.execute(sql, rec)

print("✅ CRUD helpers ready.")



✅ CRUD helpers ready.


In [37]:
# UI: chọn customer_id/year_month từ DB (fallback CSV)
import ipywidgets as w
from IPython.display import display, clear_output
from sqlalchemy import text

cust_dd = w.Dropdown(options=[], description="Customer")
ym_dd = w.Dropdown(options=[], description="Year-Month")
refresh_btn = w.Button(description="Tải danh sách", button_style="info")
load_btn = w.Button(description="Nạp hồ sơ", button_style="primary")
preview_out = w.Output(layout={"border":"1px solid #ccc"})

_customer_row = {"customer_id": None, "year_month": None}


def _load_lists(_):
    options_c = []
    options_ym = []
    if db_available():
        with engine.connect() as conn:
            dfc = pd.read_sql(text("SELECT DISTINCT customer_id FROM features_monthly ORDER BY customer_id ASC LIMIT 200"), conn)
            dfm = pd.read_sql(text("SELECT DISTINCT year_month FROM features_monthly ORDER BY year_month DESC LIMIT 12"), conn)
            options_c = list(dfc["customer_id"].astype(int).tolist())
            options_ym = list(dfm["year_month"].astype(str).tolist())
    else:
        dfc = pd.read_csv(os.path.join(DATA_DIR, "features_monthly.csv"))
        options_c = sorted(dfc["customer_id"].astype(int).unique().tolist())[:200]
        options_ym = sorted(dfc["year_month"].astype(str).unique().tolist())[::-1][:12]
    cust_dd.options = options_c
    ym_dd.options = options_ym


def _load_profile(_):
    cid = int(cust_dd.value)
    ym = str(ym_dd.value)
    if db_available():
        with engine.connect() as conn:
            sql = text("""
                SELECT f.*, l.label_interest
                FROM features_monthly f
                LEFT JOIN labels l USING (customer_id, year_month)
                WHERE f.customer_id = :cid AND f.year_month = :ym
                LIMIT 1
            """)
            row = pd.read_sql(sql, conn, params={"cid": cid, "ym": ym})
    else:
        dfc = pd.read_csv(os.path.join(DATA_DIR, "features_monthly.csv"))
        dfl = pd.read_csv(os.path.join(DATA_DIR, "labels.csv"))
        row = dfc.merge(dfl, on=["customer_id","year_month"], how="left")
        row = row[(row["customer_id"]==cid) & (row["year_month"]==ym)].head(1)
    with preview_out:
        clear_output()
        if row is None or row.empty:
            print("❌ Không tìm thấy hồ sơ.")
        else:
            display(row)
            _customer_row["customer_id"] = int(row.iloc[0]["customer_id"]) 
            _customer_row["year_month"] = str(row.iloc[0]["year_month"]) 
            print("✅ Đã nạp hồ sơ.")

refresh_btn.on_click(_load_lists)
load_btn.on_click(_load_profile)

display(w.HBox([refresh_btn, load_btn]))
display(w.HBox([cust_dd, ym_dd]))
display(preview_out)

print("ℹ️ Bấm 'Tải danh sách' → chọn customer/year_month → 'Nạp hồ sơ'.")



Output(layout=Layout(border_bottom='1px solid #ccc', border_left='1px solid #ccc', border_right='1px solid #cc…

ℹ️ Bấm 'Tải danh sách' → chọn customer/year_month → 'Nạp hồ sơ'.


In [38]:
# Affordability & Plan proposal (deterministic core)
from datetime import date, timedelta

def affordability_from_row(row: dict, horizon_days: int, goal_amount: int | None, target_months: int | None) -> dict:
    """
    Tính weekly_cap_save và đề xuất mức tiết kiệm tuần phù hợp dựa trên dữ liệu khách.
    - income_net ≈ income (fallback)
    - fixed_bills ≈ loan (fallback)
    - variable_spend ≈ spend (fallback)
    """
    income_net = int(row.get("income_net_month", row.get("income", 0)) or 0)
    fixed_bills = int(row.get("fixed_bills_month", row.get("loan", 0)) or 0)
    variable_spend = int(row.get("variable_spend_month", row.get("spend", 0)) or 0)

    base_monthly_surplus = max(0, income_net - fixed_bills - int(0.8 * variable_spend))
    # trần tuần ~ 25% surplus tháng (chừa dư địa phát sinh)
    weekly_cap_save = max(0, int(base_monthly_surplus * 0.25))

    # nếu người dùng đưa ra mục tiêu dài hạn
    recommended_weekly = weekly_cap_save
    feasibility = "unknown"
    rationale = []

    if goal_amount and target_months:
        needed_per_week = int(goal_amount / (target_months * 4.0))
        if needed_per_week <= weekly_cap_save:
            feasibility = "ok"
            recommended_weekly = needed_per_week
            rationale.append(f"Cần {needed_per_week:,}đ/tuần để đạt mục tiêu trong {target_months} tháng, trong trần {weekly_cap_save:,}đ/tuần.")
        else:
            feasibility = "adjust"
            recommended_weekly = weekly_cap_save
            rationale.append(f"Mục tiêu yêu cầu khoảng {needed_per_week:,}đ/tuần nhưng trần khả dụng chỉ {weekly_cap_save:,}đ/tuần. Đề xuất kéo dài thời gian hoặc giảm số tiền.")
    else:
        feasibility = "ok" if weekly_cap_save > 0 else "adjust"
        rationale.append("Tạm tính theo trần tuần từ dư thặng tháng.")

    return {
        "weekly_cap_save": weekly_cap_save,
        "recommended_weekly_save": recommended_weekly,
        "feasibility": feasibility,
        "reasons": rationale
    }


def propose_week_plan(row: dict, start_date: date, horizon_days: int, weekly_save: int) -> list[dict]:
    """
    Sinh kế hoạch 7/14 ngày theo số tiền weekly_save. 2-3 task/ngày, áp dụng special_days đơn giản.
    """
    tasks_per_day = max(2, min(3, weekly_save // 200000 if weekly_save else 2))
    per_day_save = max(0, int(weekly_save / (horizon_days if horizon_days >= 7 else 7)))

    # special_days đơn giản từ row: payday_dom_list, bill_dom_list, events_dom_list
    payday_dom_str = str(row.get("payday_dom", "") or "").replace(",",";")
    bills = str(row.get("bill_dom_list", "") or "")
    events = str(row.get("events_dom_list", "") or "")

    def parse_dom_list(s):
        out = []
        for token in [t.strip() for t in s.split(";") if t.strip()]:
            try:
                if ":" in token:
                    dom, rest = token.split(":", 1)
                    out.append(int(dom))
                else:
                    out.append(int(token))
            except: 
                pass
        return out

    payday_doms = parse_dom_list(payday_dom_str)

    plan = []
    cur = start_date
    for _ in range(horizon_days):
        day_tasks = []
        # payday ưu tiên chuyển tiền
        if cur.day in payday_doms:
            day_tasks.append(f"Nhận lương → chuyển ngay {min(per_day_save*2, max(per_day_save, 100000)):,}đ vào quỹ")
        # tiết kiệm theo ngày
        if per_day_save:
            day_tasks.append(f"Giữ tiết kiệm ngày: {per_day_save:,}đ (cắt bớt ăn vặt/giải trí)")
        # một hành động hành vi nhỏ
        day_tasks.append("Không mua trà sữa/cafe hôm nay; đi bộ 15 phút")
        plan.append({"date": cur.isoformat(), "tasks": [t.replace(",",".") for t in day_tasks]})
        cur += timedelta(days=1)
    return plan

print("✅ affordability() & propose_week_plan() ready.")



✅ affordability() & propose_week_plan() ready.


In [ ]:
# UI: Nhập mong muốn, đề xuất kế hoạch 7/14 ngày, xác nhận & lưu
import ipywidgets as w
from IPython.display import display, clear_output
from datetime import date
from sqlalchemy import text

wish_text = w.Text(placeholder="Ví dụ: muốn dành 15 triệu trong 6 tháng để mua điện thoại")
amount_in = w.IntText(description="Số tiền (đ)", value=15000000)
months_in = w.IntSlider(description="Thời gian (tháng)", min=1, max=36, value=6)
horizon_dd2 = w.RadioButtons(options=[7,14], description="Horizon", value=7)
persona_rb = w.ToggleButtons(options=[("Mentor","mentor"),("Angry mom","angry_mom"),("Banter","banter")], description="Persona")
propose_btn = w.Button(description="Đề xuất kế hoạch", button_style="primary")
confirm_btn = w.Button(description="Đồng ý & lưu", button_style="success")
change_btn = w.Button(description="Muốn chỉnh (regen)", button_style="warning")
feedback_in = w.Text(placeholder="Ví dụ: giảm tiết kiệm tuần xuống 400k và thêm 1 ngày nghỉ chi tiêu")

proposal_out = w.Output(layout={"border":"1px solid #ccc"})

_current_plan = {"plan": None, "days": None, "plan_id": None}


def _propose(_):
    proposal_out.clear_output()
    if not _customer_row.get("customer_id"):
        with proposal_out:
            print("❌ Chưa nạp hồ sơ khách hàng.")
        return
    cid = int(_customer_row["customer_id"])
    ym = str(_customer_row["year_month"]) or None
    persona = persona_rb.value
    horizon = int(horizon_dd2.value)

    # Lấy row lại để tính chính xác
    if db_available():
        with engine.connect() as conn:
            sql = text("""
                SELECT f.*, l.label_interest
                FROM features_monthly f
                LEFT JOIN labels l USING (customer_id, year_month)
                WHERE f.customer_id = :cid AND f.year_month = :ym
                LIMIT 1
            """)
            row_df = pd.read_sql(sql, conn, params={"cid": cid, "ym": ym})
    else:
        dfc = pd.read_csv(os.path.join(DATA_DIR, "features_monthly.csv"))
        dfl = pd.read_csv(os.path.join(DATA_DIR, "labels.csv"))
        row_df = dfc.merge(dfl, on=["customer_id","year_month"], how="left")
        row_df = row_df[(row_df["customer_id"]==cid) & (row_df["year_month"]==ym)].head(1)

    if row_df is None or row_df.empty:
        with proposal_out:
            print("❌ Không tìm thấy hồ sơ để đề xuất.")
        return

    row = row_df.iloc[0].to_dict()

    af = affordability_from_row(row, horizon_days=horizon, goal_amount=int(amount_in.value or 0), target_months=int(months_in.value or 0))
    weekly = int(af["recommended_weekly_save"])

    # Ưu tiên gọi LLM + validate schema; fallback deterministic
    try:
        proposal = llm_generate_plan_proposal_cached(persona, wish_text.value or "", row, horizon, af["weekly_cap_save"], weekly)
        days = [d.model_dump() for d in proposal.week_plan]
        weekly = int(proposal.recommended_weekly_save)
        supervision_note = proposal.supervision_note
        reasons = proposal.reasons
    except Exception:
        start_d = date.today()
        days = propose_week_plan(row, start_d, horizon, weekly)
        supervision_note = "Tôi sẽ giám sát tuần này. Đạt → lặp lại; Không đạt → điều chỉnh."
        reasons = af["reasons"]

    plan = {
        "customer_id": cid,
        "year_month": ym,
        "persona": persona,
        "horizon_days": horizon,
        "goal_text": wish_text.value or "",
        "goal_amount": int(amount_in.value or 0),
        "target_date": date.today().replace(day=28) + pd.DateOffset(months=int(months_in.value or 0)),
        "weekly_save": weekly,
        "feasibility": af["feasibility"],
        "reasons": json.dumps(reasons, ensure_ascii=False),
        "supervision_note": supervision_note,
        "committed_by": persona,
    }

    _current_plan["plan"] = plan
    _current_plan["days"] = days

    with proposal_out:
        print(f"Đề xuất cho customer {cid} ({ym}), persona={persona}")
        print(f"Feasibility: {af['feasibility']} | weekly_save đề xuất: {weekly:,}đ/tuần")
        print("Lý do:")
        for r in reasons:
            print(" -", r)
        print("\nKế hoạch:")
        for d in days:
            print(d["date"], ": ", " | ".join(d["tasks"]))
        print("\nBạn có muốn lưu kế hoạch này không? (Đồng ý & lưu / Muốn chỉnh)")


def _confirm(_):
    if not _current_plan.get("plan"):
        with proposal_out:
            print("❌ Chưa có kế hoạch để lưu.")
        return
    if not db_available():
        with proposal_out:
            print("ℹ️ DB không sẵn sàng, bỏ qua bước lưu.")
        return
    with engine.begin() as conn:
        plan = _current_plan["plan"].copy()
        # Chuyển target_date về string YYYY-MM-DD
        td = plan.get("target_date")
        if hasattr(td, "strftime"):
            plan["target_date"] = td.strftime("%Y-%m-%d")
        plan_id = db_insert_persona_plan(conn, plan, _current_plan["days"]) 
        _current_plan["plan_id"] = plan_id
        db_upsert_chat(conn, customer_id=plan["customer_id"], role="assistant", text_msg="Plan committed", payload={"plan_id": plan_id}, plan_id=plan_id)
    with proposal_out:
        print("✅ Đã lưu kế hoạch. plan_id=", _current_plan["plan_id"]) 


def _diff_plans(old_days: list, new_days: list) -> list:
    by_date = {d["date"]: d for d in old_days}
    diffs = []
    for d in new_days:
        od = by_date.get(d["date"]) or {"tasks": []}
        if od["tasks"] != d["tasks"]:
            diffs.append({
                "date": d["date"],
                "old": od.get("tasks", []),
                "new": d["tasks"]
            })
    return diffs


def _regen(_):
    if not _current_plan.get("plan"):
        with proposal_out:
            print("❌ Chưa có kế hoạch để chỉnh.")
        return
    persona = persona_rb.value
    horizon = int(horizon_dd2.value)

    # Lấy lại hồ sơ
    cid = int(_customer_row["customer_id"]) if _customer_row.get("customer_id") else None
    ym = str(_customer_row["year_month"]) if _customer_row.get("year_month") else None
    if cid is None:
        with proposal_out:
            print("❌ Chưa nạp hồ sơ.")
        return
    if db_available():
        with engine.connect() as conn:
            sql = text("""
                SELECT f.*, l.label_interest
                FROM features_monthly f
                LEFT JOIN labels l USING (customer_id, year_month)
                WHERE f.customer_id = :cid AND f.year_month = :ym
                LIMIT 1
            """)
            row_df = pd.read_sql(sql, conn, params={"cid": cid, "ym": ym})
    else:
        dfc = pd.read_csv(os.path.join(DATA_DIR, "features_monthly.csv"))
        dfl = pd.read_csv(os.path.join(DATA_DIR, "labels.csv"))
        row_df = dfc.merge(dfl, on=["customer_id","year_month"], how="left")
        row_df = row_df[(row_df["customer_id"]==cid) & (row_df["year_month"]==ym)].head(1)
    if row_df is None or row_df.empty:
        with proposal_out:
            print("❌ Không tìm thấy hồ sơ.")
        return
    row = row_df.iloc[0].to_dict()

    # Giữ trần tuần cũ làm tham chiếu, nhưng cho phép user feedback ảnh hưởng nội dung
    af = affordability_from_row(row, horizon_days=horizon, goal_amount=int(amount_in.value or 0), target_months=int(months_in.value or 0))
    weekly = int(af["recommended_weekly_save"])

    # Ghép feedback vào user_goal_text
    goal_text = (wish_text.value or "")
    fb = feedback_in.value.strip()
    user_goal_text = f"{goal_text} | feedback: {fb}" if fb else goal_text

    try:
        proposal = llm_generate_plan_proposal(persona, user_goal_text, row, horizon, af["weekly_cap_save"], weekly)
        new_days = [d.model_dump() for d in proposal.week_plan]
        weekly = int(proposal.recommended_weekly_save)
        supervision_note = proposal.supervision_note
        reasons = proposal.reasons
    except Exception:
        from datetime import date
        new_days = propose_week_plan(row, date.today(), horizon, weekly)
        supervision_note = "Tôi sẽ giám sát tuần này. Đạt → lặp lại; Không đạt → điều chỉnh."
        reasons = af["reasons"]

    old_days = _current_plan.get("days") or []
    diffs = _diff_plans(old_days, new_days)
    _current_plan["days"] = new_days
    _current_plan["plan"]["weekly_save"] = weekly
    _current_plan["plan"]["supervision_note"] = supervision_note
    _current_plan["plan"]["reasons"] = json.dumps(reasons, ensure_ascii=False)

    with proposal_out:
        print("🔁 Kế hoạch đã điều chỉnh.")
        if not diffs:
            print("(Không có thay đổi so với bản trước)")
        else:
            print("Các thay đổi:")
            for d in diffs:
                print(f" - {d['date']}")
                print("   cũ:", " | ".join(d["old"]))
                print("   mới:", " | ".join(d["new"]))

propose_btn.on_click(_propose)
confirm_btn.on_click(_confirm)
change_btn.on_click(_regen)

display(w.VBox([
    w.HTML("<b>Mong muốn</b>"),
    wish_text,
    feedback_in,
    w.HBox([amount_in, months_in, horizon_dd2]),
    persona_rb,
    w.HBox([propose_btn, confirm_btn, change_btn]),
    proposal_out
]))



In [40]:
# Prompt + JSON schema (pydantic) cho plan proposal
from pydantic import BaseModel, Field, ValidationError, field_validator
from typing import List, Optional

class Adjustment(BaseModel):
    amount: Optional[int] = Field(None, description="Số tiền điều chỉnh (VND)")
    category: Optional[str] = Field(None, description="Danh mục điều chỉnh")
    when: str = Field("today", pattern="^(today|tomorrow)$")

class PlanDay(BaseModel):
    date: str
    tasks: List[str]

class PlanProposal(BaseModel):
    feasibility: str = Field(pattern="^(ok|adjust)$")
    weekly_cap_save: int
    recommended_weekly_save: int
    reasons: List[str]
    proposal: dict
    week_plan: List[PlanDay]
    supervision_note: str
    confirm_question: str

    @field_validator("week_plan")
    @classmethod
    def limit_length(cls, v: List[PlanDay]):
        if not (7 <= len(v) <= 14):
            raise ValueError("week_plan phải có 7 hoặc 14 ngày")
        return v

STYLEBOOK = {
    "mentor": {
        "tone": "Tôn trọng, động viên, luôn có con số và lý do ngắn.",
        "forbid": ["miệt thị", "xúc phạm", "ra lệnh vô căn cứ"],
        "closing": "Bạn thấy ổn chứ?"
    },
    "angry_mom": {
        "tone": "Nghiêm khắc nhưng lịch sự, không xúc phạm; luôn đưa lựa chọn thay thế.",
        "forbid": ["coi thường", "đe doạ"],
        "closing": "Chốt vậy được không?"
    },
    "banter": {
        "tone": "Hóm hỉnh nhẹ, không châm chọc cá nhân; vẫn phải có số liệu.",
        "forbid": ["mỉa mai cá nhân", "chê bai cơ thể"],
        "closing": "Ok mình chơi kèo này nhé?"
    }
}

SYSTEM_PROMPT = (
    "Bạn là trợ lý tài chính theo persona, nhiệm vụ: \n"
    "1) Kiểm tra khả thi dựa vào dữ liệu tài chính của người dùng (thu nhập ròng, chi phí cố định, chi tiêu biến đổi, sự kiện).\n"
    "2) Nếu khả thi: đề xuất kế hoạch 7 hoặc 14 ngày, mỗi ngày 2–4 task đo lường được, tổng weekly_save ≤ trần khả thi.\n"
    "3) Nếu chưa khả thi: đề xuất điều chỉnh (giảm số tiền/giãn thời gian) kèm lý do ngắn.\n"
    "4) Persona chỉ là giọng điệu; TUYỆT ĐỐI không xúc phạm/công kích.\n"
    "5) Chỉ trả về JSON đúng schema PlanProposal (không thêm chữ nào ngoài JSON).\n"
)

print("✅ Prompt & schema sẵn sàng.")



✅ Prompt & schema sẵn sàng.


In [41]:
# Hàm gọi LLM sinh PlanProposal theo SYSTEM_PROMPT + STYLEBOOK, validate bằng Pydantic

def llm_generate_plan_proposal(persona: str, user_goal_text: str, row: dict, horizon_days: int, weekly_cap_save: int, recommended_weekly: int):
    persona = persona if persona in STYLEBOOK else "mentor"
    tone = STYLEBOOK[persona]["tone"]
    closing = STYLEBOOK[persona]["closing"]

    payload = {
        "persona": persona,
        "tone": tone,
        "user_goal": user_goal_text,
        "horizon_days": horizon_days,
        "weekly_cap_save": weekly_cap_save,
        "recommended_weekly_save": recommended_weekly,
        "row_summary": {
            "income": int(row.get("income_net_month", row.get("income", 0)) or 0),
            "fixed_bills": int(row.get("fixed_bills_month", row.get("loan", 0)) or 0),
            "variable_spend": int(row.get("variable_spend_month", row.get("spend", 0)) or 0),
        },
        "style_closing": closing
    }

    if 'gemini_model' in globals() and gemini_model is not None:
        try:
            import google.generativeai as genai
            mdl = genai.GenerativeModel(
                model_name=os.environ.get("GEMINI_MODEL", "gemini-2.0-flash"),
                system_instruction=SYSTEM_PROMPT,
                generation_config={
                    "temperature": 0.5,
                    "response_mime_type": "application/json"
                }
            )
            resp = mdl.generate_content(json.dumps(payload, ensure_ascii=False))
            text = resp.candidates[0].content.parts[0].text if resp and resp.candidates else ""
            obj = json.loads(text)
            validated = PlanProposal(**obj)
            return validated
        except Exception as e:
            # fallback deterministic
            pass

    # fallback deterministic: xây dựng từ propose_week_plan
    from datetime import date
    days = propose_week_plan(row, date.today(), horizon_days, recommended_weekly)
    deterministic = {
        "feasibility": "ok" if recommended_weekly>0 else "adjust",
        "weekly_cap_save": weekly_cap_save,
        "recommended_weekly_save": recommended_weekly,
        "reasons": [
            "Fallback deterministic do LLM không sẵn sàng."
        ],
        "proposal": {
            "target_amount": None,
            "target_date": None
        },
        "week_plan": days,
        "supervision_note": "Tôi sẽ giám sát tuần này. Đạt → lặp lại; Không đạt → điều chỉnh.",
        "confirm_question": "Bạn đồng ý kế hoạch này không?"
    }
    return PlanProposal(**deterministic)

print("✅ LLM proposal generator ready.")



✅ LLM proposal generator ready.


In [42]:
# UI: Ghi nhật ký chat & chi tiêu, điều chỉnh kế hoạch theo chi tiêu phát sinh
import ipywidgets as w
from IPython.display import display, clear_output
from datetime import date

chat_user = w.Text(placeholder="Nhập tin nhắn để log vào DB (tuỳ chọn)")
chat_btn = w.Button(description="Ghi nhật ký chat", button_style="info")

spend_text = w.Text(placeholder="Ví dụ: uống trà sữa 60k")
spend_btn = w.Button(description="Ghi chi tiêu & điều chỉnh", button_style="warning")
log_out = w.Output(layout={"border":"1px solid #ccc"})


def _log_chat(_):
    msg = chat_user.value.strip()
    if not msg:
        return
    with log_out:
        print(f"👤 user: {msg}")
    if db_available() and _customer_row.get("customer_id"):
        with engine.begin() as conn:
            db_upsert_chat(conn, customer_id=int(_customer_row["customer_id"]), role="user", text_msg=msg, payload={"ym": _customer_row.get("year_month")}, plan_id=_current_plan.get("plan_id"))
    chat_user.value = ""


def _adjust_plan_for_spend(amount: int):
    """Đơn giản: thêm task bù vào ngày hôm sau, hoặc hôm nay nếu chưa có plan.
    """
    if not _current_plan.get("days"):
        return [], []
    old_days = [dict(d) for d in _current_plan["days"]]
    days = [dict(date=d["date"], tasks=list(d["tasks"])) for d in _current_plan["days"]]
    # chọn ngày bù: hôm nay (nếu có trong plan) hoặc ngày đầu tiên kế tiếp
    today_str = date.today().isoformat()
    idx = next((i for i,d in enumerate(days) if d["date"] >= today_str), 0)
    bump = f"Bù lại {amount:,}đ bằng cách cắt ăn vặt/giải trí".replace(",",".")
    days[idx]["tasks"].insert(0, bump)
    _current_plan["days"] = days
    return old_days, days


def _spend(_):
    text_msg = spend_text.value.strip()
    if not text_msg:
        return
    # Parse amount & category từ cell B (đã có _extract_amount, _guess_category, add_spend_event)
    amt = _extract_amount(text_msg)
    cat = _guess_category(text_msg)
    rec = add_spend_event(text_msg)

    # Log DB
    if db_available() and _customer_row.get("customer_id"):
        with engine.begin() as conn:
            db_insert_spend_event(conn, customer_id=int(_customer_row["customer_id"]), text_msg=text_msg, amount=amt, category=cat, source="user", plan_id=_current_plan.get("plan_id"))
            db_upsert_chat(conn, customer_id=int(_customer_row["customer_id"]), role="user", text_msg=f"spend: {text_msg}", payload={"amount": amt, "category": cat}, plan_id=_current_plan.get("plan_id"))

    advice = None
    if _current_plan.get("plan"):
        old_days, new_days = adjust_plan_with_spend_optimal(int(amt or 0))
        diffs = _diff_plans(old_days, new_days) if old_days else []
        advice = f"Đã thêm task bù {amt:,}đ vào kế hoạch.".replace(",",".") if amt else "Đã ghi nhận chi tiêu."
    else:
        advice = "Đã ghi nhận chi tiêu. Hãy tạo kế hoạch trước để mình giúp điều chỉnh."

    with log_out:
        print(f"🧾 spend: {text_msg} (amt={amt}, cat={cat})")
        print("🤖 advice:", advice)
        if _current_plan.get("plan"):
            if not diffs:
                print("(Không có thay đổi kế hoạch hoặc chưa có plan)")
            else:
                print("Các thay đổi trong plan:")
                for d in diffs:
                    print(f" - {d['date']}")
                    print("   cũ:", " | ".join(d["old"]))
                    print("   mới:", " | ".join(d["new"]))

    # Log assistant advice
    if db_available() and _customer_row.get("customer_id"):
        with engine.begin() as conn:
            db_upsert_chat(conn, customer_id=int(_customer_row["customer_id"]), role="assistant", text_msg=advice, payload={"type":"adjust_on_spend","amount": amt, "category": cat}, plan_id=_current_plan.get("plan_id"))

    spend_text.value = ""

chat_btn.on_click(_log_chat)
spend_btn.on_click(_spend)

display(w.VBox([
    w.HTML("<b>Nhật ký & Điều chỉnh theo chi tiêu</b>"),
    w.HBox([chat_user, chat_btn]),
    w.HBox([spend_text, spend_btn]),
    log_out
]))



In [43]:
# Cache LLM proposal + điều chỉnh kế hoạch tối ưu theo chi tiêu
import hashlib

PROPOSAL_CACHE: dict[str, dict] = {}

def _cache_key(persona: str, user_goal_text: str, horizon_days: int, weekly_cap_save: int, recommended_weekly: int, row: dict) -> str:
    row_summary = {
        "income": int(row.get("income_net_month", row.get("income", 0)) or 0),
        "fixed_bills": int(row.get("fixed_bills_month", row.get("loan", 0)) or 0),
        "variable_spend": int(row.get("variable_spend_month", row.get("spend", 0)) or 0),
    }
    payload = json.dumps({
        "persona": persona,
        "goal": user_goal_text,
        "horizon": horizon_days,
        "cap": weekly_cap_save,
        "rec": recommended_weekly,
        "row": row_summary,
    }, ensure_ascii=False, sort_keys=True)
    return hashlib.sha256(payload.encode("utf-8")).hexdigest()


def llm_generate_plan_proposal_cached(persona: str, user_goal_text: str, row: dict, horizon_days: int, weekly_cap_save: int, recommended_weekly: int):
    key = _cache_key(persona, user_goal_text, horizon_days, weekly_cap_save, recommended_weekly, row)
    if key in PROPOSAL_CACHE:
        obj = PROPOSAL_CACHE[key]
        try:
            return PlanProposal(**obj)
        except Exception:
            pass
    proposal = llm_generate_plan_proposal(persona, user_goal_text, row, horizon_days, weekly_cap_save, recommended_weekly)
    try:
        PROPOSAL_CACHE[key] = proposal.model_dump()
    except Exception:
        pass
    return proposal


def adjust_plan_with_spend_optimal(amount: int):
    """Phân bổ bù vào nhiều ngày còn lại, tổng bù = amount, ưu tiên không vượt quá ~per_day_save.
    """
    if amount is None or amount <= 0 or not _current_plan.get("days"):
        return [], []
    days = [dict(date=d["date"], tasks=list(d["tasks"])) for d in _current_plan["days"]]
    old_days = [dict(date=d["date"], tasks=list(d["tasks"])) for d in _current_plan["days"]]

    horizon = int(_current_plan["plan"].get("horizon_days", len(days))) if _current_plan.get("plan") else len(days)
    weekly = int(_current_plan["plan"].get("weekly_save", 0)) if _current_plan.get("plan") else 0
    per_day_target = max(1, int(max(1, weekly) / max(7, horizon))) if weekly else max(1, int(amount / max(1, len(days))))

    # Chọn tập ngày còn lại kể từ hôm nay
    today_str = date.today().isoformat()
    rem_idx = [i for i,d in enumerate(days) if d["date"] >= today_str]
    if not rem_idx:
        rem_idx = list(range(len(days)))

    remaining = int(amount)
    # phân bổ vòng tròn qua các ngày còn lại đến khi hết
    ptr = 0
    while remaining > 0 and rem_idx:
        i = rem_idx[ptr % len(rem_idx)]
        chunk = min(per_day_target, remaining)
        bump = f"Bù lại {chunk:,}đ do chi tiêu phát sinh".replace(",",".")
        days[i]["tasks"].insert(0, bump)
        remaining -= chunk
        ptr += 1

    _current_plan["days"] = days
    return old_days, days

print("✅ Cache & optimal adjust ready.")



✅ Cache & optimal adjust ready.


In [44]:
# .env config loader (không ghi file, chỉ nạp nếu có)
try:
    from dotenv import load_dotenv
    load_dotenv()
    print("✅ Loaded .env if present.")
except Exception as e:
    print("ℹ️ python-dotenv not installed; bỏ qua.")

# Kiểm thử nhanh parsing số và validate JSON output

def test_parsers_and_schema():
    tests = [
        ("trà sữa 60k", 60000),
        ("ăn buffet 230k", 230000),
        ("mua áo 1.2m", 1200000),
        ("coffee 15,000", 15000),
    ]
    ok = True
    for s, expect in tests:
        got = _extract_amount(s)
        if got != expect:
            print("❌ parse:", s, "=>", got, "!=", expect)
            ok = False
    if ok:
        print("✅ amount parser passed.")

    # Validate mẫu PlanProposal tối thiểu (7 ngày)
    sample = {
        "feasibility": "ok",
        "weekly_cap_save": 700000,
        "recommended_weekly_save": 600000,
        "reasons": ["demo"],
        "proposal": {"target_amount": 15000000, "target_date": "2026-02-01"},
        "week_plan": [{"date":"2025-09-10","tasks":["Giữ tiết kiệm ngày 100.000đ"]} for _ in range(7)],
        "supervision_note": "Tôi sẽ giám sát tuần này. Đạt → lặp lại; Không đạt → điều chỉnh.",
        "confirm_question": "Bạn đồng ý không?"
    }
    try:
        PlanProposal(**sample)
        print("✅ schema validation passed.")
    except ValidationError as e:
        print("❌ schema validation:", e)

# Chạy test nhanh (có thể tắt nếu không cần)
test_parsers_and_schema()



ℹ️ python-dotenv not installed; bỏ qua.
❌ parse: mua áo 1.2m => 12000000 != 1200000
✅ schema validation passed.
